In [18]:
import numpy as np
import os
from abc import abstractmethod

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [13]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))
    x_test = np.load(os.path.join(folder, 'x_test.npy'))
    y_test = np.load(os.path.join(folder, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [9]:
def assert_preds_correct(your_preds, sklearn_preds) -> bool:
    return np.abs(your_preds - sklearn_preds).sum() == 0

In [10]:
def assert_probs_correct(your_probs, sklearn_probs) -> bool:
    return np.abs(your_probs - sklearn_probs).mean() < 1e-3

In [11]:
# Не изменяйте код этого класса!
class NaiveBayes:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.params = dict()

    # --- PREDICTION ---

    def predict(self, x, return_probs=False):
        """
        x - np.array размерности [N, dim],
        где N - количество экземпляров данных,
        dim -размерность одного экземпляра (количество признаков).

        Возвращает np.array размерности [N], содержащий номера классов для
        соответствующих экземпляров.
        """
        preds = []
        for sample in x:
            preds.append(
                self.predict_single(sample, return_probs=return_probs)
            )

        if return_probs:
            return np.array(preds, dtype='float32')

        return np.array(preds, dtype='int32')

    # Совет: вниманительно изучите файл подсказок к данной лабораторной
    # и сопоставьте код с описанной математикой байесовского классификатора.
    def predict_single(self, x, return_probs=False) -> int:
        """
        Делает предсказание для одного экземпляра данных.

        x - np.array размерности dim.

        Возвращает номер класса, которому принадлежит x.
        """
        assert len(x.shape) == 1, f'Expected a vector, but received a tensor of shape={x.shape}'
        marginal_prob = self.compute_marginal_probability(x)  # P(x) - безусловная вероятность появления x

        probs = []
        for c in range(self.n_classes):                 # c - номер класса
            prior = self.compute_prior(c)               # P(c) - априорная вероятность (вероятность появления класса)
            likelihood = self.compute_likelihood(x, c)  # P(x|c) - вероятность появления x в предположении, что он принаждлежит c

            # Используем теорему Байесса для просчёта условной вероятности P(c|x)
            # P(c|x) = P(c) * P(x|c) / P(x)
            prob = prior * likelihood / marginal_prob
            probs.append(prob)

        if return_probs:
            return probs

        return np.argmax(probs)

    # Вычисляет P(x) - безусловная вероятность появления x.
    @abstractmethod
    def compute_marginal_probability(self, x) -> float:
        pass

    # Вычисляет P(c) - априорная вероятность появления класса c.
    @abstractmethod
    def compute_prior(self, c) -> float:
        pass

    # Вычисляет P(x|c) - вероятность наблюдения экземпляра x в предположении, что он принаждлежит c.
    @abstractmethod
    def compute_likelihood(self, x, c) -> float:
        pass

    # --- FITTING ---

    def fit(self, x, y):
        self._estimate_prior(y)
        self._estimate_params(x, y)

    @abstractmethod
    def _estimate_prior(self, y):
        pass

    @abstractmethod
    def _estimate_params(self, x, y):
        pass

## 1. Наивный классификатор Байеса: гауссово распределение

Напишите недостающий код, создайте и обучите модель.

Пункты оценки:
1. совпадение предсказанных классов с оными у модели sklearn. Для проверки совпадения используйте функцию `assert_preds_correct`.
2. совпадение значений предсказанных вероятностей принадлежности классами с оными у модели sklearn. Значения вероятностей считаются равными, если функция `assert_probs_correct` возвращает True.

In [15]:
x_train, y_train, x_test, y_test = load_data('/gauss')

In [22]:
# P(C_k|x) = P(x|theta) * P(C_k) / P(x)

class NaiveGauss(NaiveBayes):
    def compute_marginal_probability(self, x) -> float:
        # Для просчёта безусловной вероятности используйте
        # методы compute_prior и compute_likelihood.
        # Напишите свой код здесь
        marginal_prob = 0.0
        for c in range(self.n_classes):
            prior = self.compute_prior(c)
            likelihood = self.compute_likelihood(x, c)
            marginal_prob += prior * likelihood
        return marginal_prob

    def compute_prior(self, c) -> float:
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        return self.params['prior'][c]

    def compute_likelihood(self, x, c) -> float:
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        mean = self.params['mean'][c]
        var = self.params['var'][c]
        likelihood = np.prod(1 / np.sqrt(2 * np.pi * var) * np.exp(-0.5 * ((x - mean) ** 2) / var))
        return likelihood

    # --- FITTING ---

    def _estimate_prior(self, y):
        # Значения априорных вероятностей сохраните в `params` с ключом 'prior'
        class_counts = np.bincount(y, minlength=self.n_classes)
        self.params['prior'] = class_counts / len(y)

    def _estimate_params(self, x, y):
        self.params['mean'] = []
        self.params['var'] = []
        for c in range(self.n_classes):
            x_c = x[y == c]
            mean_c = x_c.mean(axis=0)
            var_c = x_c.var(axis=0)
            self.params['mean'].append(mean_c)
            self.params['var'].append(var_c)
        self.params['mean'] = np.array(self.params['mean'])
        self.params['var'] = np.array(self.params['var'])

In [23]:
# Создайте и обучите модель
our_nb = NaiveGauss(n_classes=np.unique(y_train).shape[0])
our_nb.fit(x_train, y_train)

In [24]:
# Оцените качество модели
our_preds = our_nb.predict(x_test)
our_probs = our_nb.predict(x_test, return_probs=True)

accuracy = accuracy_score(y_test, our_preds)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00


In [27]:
# Сравните вашу модель с аналогом sklearn (GaussianNB)
skl_nb = GaussianNB()
skl_nb.fit(x_train, y_train)

skl_preds = skl_nb.predict(x_test)
skl_probs = skl_nb.predict_proba(x_test)

skl_accuracy = accuracy_score(y_test, skl_preds)
print(f'Accuracy: {skl_accuracy:.2f}')

assert assert_preds_correct(our_preds, skl_preds), "Предсказанные классы нашей модели и модели sklearn НЕ совпадают!"
assert assert_probs_correct(our_probs, skl_probs), "Вероятности нашей модели и модели sklearn НЕ совпадают!"

print("Предсказанные классы и вероятности нашей модели совпадают с классами и вероятностями модели sklearn🥳")

Accuracy: 1.00
Предсказанные классы и вероятности нашей модели совпадают с классами и вероятностями модели sklearn🥳


## 2. Доп. задания (любое на выбор, опционально)

### 2.1  Упрощение наивного классификатора Байеса для гауссова распределения

Уберите из класса NaiveBayes 'лишние' вычисления и удалите код, что соответствует этим вычислениям. Под 'лишним' подразумеваются вещи, что не влияют на итоговое решение о принадлежности классу (значения вероятностей при этом могу стать некорректными, но в данном задании это допустимо).

Напишите в клетке ниже код упрощенного 'классификатора Гаусса' и убедитесь, что его ответы (не значения вероятностей) совпадают с ответами классификатора из задания 1. Для сравнения ответов используйте функцию `assert_preds_correct`.

Указание: работайте в предположении, что классы равновероятны.

Подсказка: упростить необходимо метод `predict_single`.

In [28]:
# Напишите обновленный код модели здесь
class NaiveGaussSimple(NaiveBayes):
    def compute_prior(self, c) -> float:
        return 1.0

    def compute_likelihood(self, x, c) -> float:
        mean = self.params['mean'][c]
        var = self.params['var'][c]
        likelihood = np.prod(1 / np.sqrt(2 * np.pi * var) * np.exp(-0.5 * ((x - mean) ** 2) / var))
        return likelihood

    def predict_single(self, x, return_probs=False) -> int:
        probs = []
        for c in range(self.n_classes):
            likelihood = self.compute_likelihood(x, c)  # P(x|c)
            probs.append(likelihood)

        return np.argmax(probs)

    def _estimate_params(self, x, y):
        self.params['mean'] = []
        self.params['var'] = []
        for c in range(self.n_classes):
            x_c = x[y == c]
            mean_c = x_c.mean(axis=0)
            var_c = x_c.var(axis=0)
            self.params['mean'].append(mean_c)
            self.params['var'].append(var_c)
        self.params['mean'] = np.array(self.params['mean'])
        self.params['var'] = np.array(self.params['var'])


In [29]:
# Создайте и обучите модель
simple_nb = NaiveGaussSimple(n_classes=np.unique(y_train).shape[0])
simple_nb.fit(x_train, y_train)

In [32]:
# Оцените качество модели
simple_nb_preds = simple_nb.predict(x_test)

simple_nb_accuracy = accuracy_score(y_test, simple_nb_preds)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00


In [33]:
# Сравните вашу модель с моделью из задания 1
assert assert_preds_correct(our_preds, simple_nb_preds), "Предсказанные классы НЕ совпадают!"

print("Предсказанные классы совпадают🥳")

Предсказанные классы совпадают🥳


In [ ]:
# Объясните в комментариях к этой клетке суть проделанных изменений: почему удаленный код является лишним?
# Раз мы предполагаем, что классы равноверояны, то мы можем убрать расчет априорных вероятностей и не учитывать их в дальнейшем,
# ведь вероятность каждго класса одинаковая и не влияет на выбор класса, который мы предсказываем
# Также был убран подсчет безусловной вероятности появления x, т.к. нам в целом важен тот класс, для которого вероятность максимальна,
# а без P(x) относительные соотношения между вероятностями сохраняются.
# Соответственно, в predict_single были убраны подсчеты априорной и безусловной вероятностей.

### 2.1  Наивный классификатор Байеса: мультиномиальное распределения

Напишите недостающий код, создайте и обучите модель.

Подсказка: в определении функции правдоподобия много факториалов. Для избежания численного переполнения посчитайте сначала логарифм функции правдоподобия (на бумаге), после примените экспоненту для получения значения вероятности.

Пункты оценки:
1. совпадение предсказанных классов с оными у модели sklearn. Для проверки совпадения используйте функцию `assert_preds_correct`.
2. совпадение значений предсказанных вероятностей принадлежности классами с оными у модели sklearn. Значения вероятностей считаются равными, если функция `assert_probs_correct` возвращает True.

Сложность: математический гений.

In [ ]:
x_train, y_train, x_test, y_test = load_data('multinomial')

In [ ]:
"""
При желании данный класс можно переписать с нуля. Изменения должны сопровождаться комментариями.
"""
class NaiveMultinomial(NaiveBayes):
    def compute_marginal_probability(self, x) -> float:
        # Для просчёта безусловной вероятности используйте
        # методы compute_prior и compute_likelihood.
        # Напишите свой код здесь
        pass

    def compute_prior(self, c) -> float:
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        # Напишите свой код здесь
        pass

    def compute_likelihood(self, x, c) -> float:
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        # Напишите свой код здесь
        pass

    # --- FITTING ---

    def _estimate_prior(self, y):
        # Значения априорных вероятностей сохраните в `params` с ключом 'prior'
        # Напишите свой код здесь
        pass

    def _estimate_params(self, x, y):
        # Напишите свой код здесь
        pass

In [ ]:
# Создайте и обучите модель

In [ ]:
# Оцените качество модели

In [ ]:
# Сравните вашу модель с аналогом sklearn (MultinomialNB)